# ASTR 119 Final Project Option #3: Astronomical Source Detection
Audrey Do and Justine Rafael

#### Import necessary packages (Justine)

In [ ]:
%matplotlib inline
import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib import rcParams
from matplotlib.patches import Ellipse

In [ ]:
rcParams['figure.figsize'] = [15., 12.]

#### Open the FITS file and read into standard 2-D numpy array (Justine)

In [ ]:
fname = "https://archive.stsci.edu/pub/hlsp/hudf12/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()
image_data = hdu_list[0].data

#### Close the FITS file (Justine)

In [ ]:
hdu_list.close()

#### Show the image (Justine)

In [ ]:
m = np.mean(image_data)
s = np.std(image_data)

plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# Background subtraction

#### Measure a spatially varying background on the image (Justine)

In [ ]:
image_data = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data)

#### Get a "global" mean and noise of the image background (Justine)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

#### Evaluate background as 2-D array, same size as original image (Justine)

In [ ]:
bkg_image = bkg.back()

#### Show the  (Justine)

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#### Evaluate the background nosie as 2-D array, same size as original image (Justine)

In [ ]:
bkg_rms = bkg.rms()

#### Show the background noise (Justine)

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#### Subtract the background (Justine)

In [ ]:
image_data_sub = image_data - bkg

# Object detection

#### Set the detection threshold to a constant value (Audrey)

In [ ]:
objects = sep.extract(image_data_sub, 1.5, err=bkg.globalrms)

n = len(objects)
print('%d objects detected'%n)

#### Plot background-subtracted image with a red ellipse for each object (Audrey)

In [ ]:
fig, ax = plt.subplots()
m = np.mean(image_data_sub)
s= np.std(image_data_sub)
im = ax.imshow(image_data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

# Aperture photometry

#### Perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects (Audrey)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(image_data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

#### Histogram the fluxes (Audrey)

In [ ]:
histogram = plt.hist(flux, bins='auto', range = [-1,2], histtype = 'bar')